In [2]:
import numpy as np
import os
import matplotlib.pyplot as plt
import freqopttest.util as util
import freqopttest.data as data
import freqopttest.kernel as kernel
import freqopttest.tst as tst
import collections
from collections import defaultdict
import freqopttest.glo as glo
import sys
import time
import os
import pandas as pd
import pickle

WARNING (theano.configdefaults): install mkl with `conda install mkl-service`: No module named 'mkl'


In [7]:
file = pd.read_pickle('/media/ak/T7/August11th2022Experiments/LinearMMDInputFiles/G_1_calendar_mfdfa_dicts_all_dates.pkl')
variables = ['n_F', 'list_H', 'list_H_intercept', 'tau', 'alpha', 'mfSpect']


/media/ak/T71/August11th2022Experiments/experimentOne/FB1 True


FileNotFoundError: [Errno 2] No such file or directory: '/media/ak/T71/August11th2022Experiments/mfdfaDataFrames/JB1_calendar_alpha.pkl'

In [ ]:

len = unpickled_Df.shape[1]
print(len)
test_results_one_dict = defaultdict(dict)
test_results_two_dict = defaultdict(dict)

if __name__ == '__main__':
    if __name__ == '__main__':
        shift = 1
        window = 5  # slide
        for start_point in range(0, (len - window - shift)):
            end_point = start_point + shift

            X = np.array(unpickled_Df.iloc[:, start_point:end_point])
            Y = np.array(unpickled_Df.iloc[:, end_point + shift:end_point + 2 * shift])
            Z = np.array(unpickled_Df.iloc[:, start_point + window:end_point + window])

            data_sample = data.TSTData(X, Y)  # data to train the model
            test_data_one = data_sample = data.TSTData(X, Z)
            test_data_two = data_sample = data.TSTData(Y, Z)

            tr, te = data_sample.split_tr_te(tr_proportion=0.9, seed=100)

            # choose the best kernel that maximizes the test power
            med = util.meddistance(tr.stack_xy())

            widths = [(med * f) for f in 2.0 ** np.linspace(-1, 4, 25)]
            list_kernels = [kernel.KGauss(w ** 2) for w in widths]

            besti, powers = tst.LinearMMDTest.grid_search_kernel(tr, list_kernels, alpha=0.01)
            # print(list_kernels)
            plt.plot(widths, powers, 'o-')
            plt.xlabel('Gaussian width')
            plt.ylabel('test power')
            plt.title('median distance = %.3g. Best width: %.3g' % (med, widths[besti]))
            plt.show()
            test_data_one = data_sample = data.TSTData(X, Z)
            test_data_two = data_sample = data.TSTData(Y, Z)

            # The actual test
            best_ker = list_kernels[besti]
            lin_mmd_test = tst.LinearMMDTest(best_ker, alpha=0.01)
            # test_result_one = lin_mmd_test.perform_test(test_data_one)
            # test_result_two = lin_mmd_test.perform_test(test_data_two)

            # Test 1 Results Dict
            test_results_one_dict[start_point]['widths'] = widths
            test_results_one_dict[start_point]['med'] = med
            test_results_one_dict[start_point]['widths'] = widths
            test_results_one_dict[start_point]['besti'] = besti
            test_results_one_dict[start_point]['powers'] = powers

            test_results_one_dict[start_point]['med_on_test_data'] = util.meddistance(test_data_one.stack_xy())
            test_results_one_dict[start_point]['test_result'] = lin_mmd_test.perform_test(test_data_one)
            test_results_one_dict[start_point]['test_variance'] = lin_mmd_test.variance(X, Z, best_ker)  # test variance
            test_results_one_dict[start_point]['two_moments'] = lin_mmd_test.two_moments(X, Z,
                                                                                         best_ker)  # test variance
            test_results_one_dict[start_point]['compute_unbiased_linear_estimator'] = lin_mmd_test.compute_stat(
                test_data_one)

            # Test 2 Results Dict
            test_results_two_dict[start_point]['test_result'] = lin_mmd_test.perform_test(test_data_two)
            test_results_two_dict[start_point]['test_variance'] = lin_mmd_test.variance(Y, Z, best_ker)
            test_results_two_dict[start_point]['med_on_test_data'] = util.meddistance(
                test_data_two.stack_xy())  # test variance
            test_results_two_dict[start_point]['two_moments'] = lin_mmd_test.two_moments(Y, Z,
                                                                                         best_ker)  # test variance
            test_results_two_dict[start_point]['compute_unbiased_linear_estimator'] = lin_mmd_test.compute_stat(
                test_data_two)

            # need a better hash file below

        pickle_out_dict_one = os.path.join(LinearMMDOutputFiles, "".join(
            (str(symbol) + "_" + str(variable) + "_shift_" + str(shift) + "_wind_" + str(window) + "_" + str(
                'linear_test') + "_ONE.pkl")))
        pickle.dump(test_results_one_dict, open(pickle_out_dict_one, 'wb'), protocol=pickle.HIGHEST_PROTOCOL)

        pickle_out_dict_two = os.path.join(LinearMMDOutputFiles, "".join(
            (str(symbol) + "_" + str(variable) + "_shift_" + str(shift) + "_wind_" + str(window) + "_" + str(
                'linear_test') + "_TWO.pkl")))
        pickle.dump(test_results_two_dict, open(pickle_out_dict_two, 'wb'), protocol=pickle.HIGHEST_PROTOCOL)

        print(test_results_one_dict)

    # print(test_results_one_dict)


In [ ]:
from multiprocessing import Pool, cpu_count

def analyze_column(start_point, end_point, shift, window):
    mmd_tester = MMDTester()
    try:
        test_results_one, test_results_two = mmd_tester.analyze(start_point, end_point, shift, window)
        col1, col2 = unpickled_df.columns[start_point], unpickled_df.columns[end_point + shift]
        result_key = f"{col1} vs {col2}"
        if result_key not in results_dict:
            results_dict[result_key] = {}
        if window not in results_dict[result_key]:
            results_dict[result_key][window] = {}
        results_dict[result_key][window][shift] = (test_results_one, test_results_two)
    except ValueError:
        pass

# Generate all possible combinations of column pairs, window sizes, and shifts
column_pairs = list(combinations(unpickled_dataframe.columns, 2))
windows = range(5, 201, 5)
shifts = range(1, 11)

# Create a list of arguments for the analyze_column function
args_list = []
for start_point, end_point in combinations(range(len(unpickled_dataframe.columns)), 2):
    for window in windows:
        for shift in shifts:
            args_list.append((start_point, end_point, shift, window))

# Use multiprocessing to analyze the data in parallel
with Pool(processes=cpu_count()) as pool:
    pool.starmap(analyze_column, args_list)

# Convert the results dictionary to a DataFrame
result_dict = {}
for col_pair, window_dict in results_dict.items():
    for window, shift_dict in window_dict.items():
        for shift, test_results in shift_dict.items():
            result_key = f"{col_pair}, window={window}, shift={shift}"
            result_dict[result_key] = test_results
result_df = pd.DataFrame.from_dict(result_dict, orient='index', columns=['Test Results 1', 'Test Results 2'])

# # Save the results to a CSV file
# result_df.to_csv('test_results.csv')
